In [ ]:
import os
import json
import pandas as pd


In [ ]:

def create_summary_files(directory, path):
    # Define the names of the summary files
    directory_name = os.path.basename(directory)
    summary_file_raw = os.path.join(path, f"{directory_name}-Raw.txt")
    summary_file = os.path.join(path, f"{directory_name}.txt")

    # Initialize strings to store file contents
    raw_content = ""
    non_raw_content = ""

    # Iterate through the files in the directory
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)
        # Check if the current item is a file and not one of the summary files
        if os.path.isfile(file_path) and file_name not in [os.path.basename(summary_file_raw), os.path.basename(summary_file)]:
            # Read the file content
            with open(file_path, 'r') as file:
                content = file.read()
                print(content)
                if "-Raw" in file_name:
                    raw_content += "Scenario: "+ file_name +"\n"
                    raw_content += content + "\n"
                else:
                    non_raw_content +="Scenario: "+ file_name +"\n"
                    non_raw_content += content + "\n"

    # Write the collected raw file contents to "Summary File Raw.txt"
    with open(summary_file_raw, 'w') as file:
        file.write(raw_content)

    # Write the collected non-raw file contents to "Summary File.txt"
    with open(summary_file, 'w') as file:
        file.write(non_raw_content)

# Example usage
directory_path =r"D:\Dissertation-GPT\5.25-NhuY\Matcha\AddLesson"  # Replace with your folder path
path = r"D:\Dissertation-GPT\5.25-NhuY\Matcha\summary test case"
create_summary_files(directory_path, path)


In [ ]:
def create_json_testcase_file(file_content,filename):
    with open(file_content, 'r') as file:
        content = file.read()
    scenarios = content.strip().split('Scenario: ')
    scenarios = [s for s in scenarios if s.strip()]  # Remove empty strings if any

    # Function to parse each scenario into a dictionary
    def parse_scenario(scenario_text):
        lines = scenario_text.strip().split('\n')
        scenario_name = lines[0].strip()        
        test_cases = []
        new_string = '\n'.join(lines[1:])
        testcase = new_string.strip().split('Test Case: ')
        print(testcase)
        for i in testcase:
            lines = i.strip().split('\n')
            test_case = {}
            for line in lines[1:]:
                if line.startswith('testSteps:'):
                    test_case['testSteps'] = []
                    continue
                if line.startswith('  - Step '):
                    test_case['testSteps'].append(line.strip())
                    continue
                if ':' in line:
                    key, value = line.split(':', 1)
                    test_case[key.strip()] = value.strip()
            test_cases.append(test_case)
            # print(scenario_name, test_case)
        return scenario_name, test_cases

    # Parse all scenarios
    parsed_scenarios = {}
    for scenario in scenarios:
        name, details = parse_scenario(scenario)
        for i in range(len(details)):
            str_num = str(i)
            element_name = name + str_num
            parsed_scenarios[element_name] = details[i]

    with open(filename, 'w') as f:
        json.dump(parsed_scenarios, f, indent=4)


In [ ]:

def update_excel_with_json_testcase_data(json_file, excel_file):
    # Load JSON data
    with open(json_file, 'r') as f:
        json_data = json.load(f)
    
    # Load Excel file
    df = pd.read_excel(excel_file)
    
    # Iterate through each row in the Excel file
    for index, row in df.iterrows():
        testcase_name = row['Test Case Name']  # Assuming column A is named 'Scenario Name'
        print(testcase_name)

        # Search for the scenario name in the JSON data
        for key, value in json_data.items():
            if value.get("testCaseName") == testcase_name:
                print(value)
                test_steps = json_data[key].get('testSteps', [])
                expected_result = json_data[key].get('expectedResult', "")
                objective = json_data[key].get('objective', [])
                test_step_str =  '\n'.join(test_steps)
                combined_info = (
                        f"Test Case Name: {testcase_name}\n"
                        f"Objective: {objective}\n"
                        f"Test Steps: { test_step_str}\n"
                        f"Expected Result: {expected_result}"
                )
                # Update the corresponding cells in the DataFrame
                df.at[index, 'Test Case'] = combined_info
                df.at[index, 'Test Steps'] = test_step_str  # Assuming column B is named 'Test Steps'
                df.at[index, 'Expected Result'] = expected_result  # Assuming column C is named 'Expected Result'
                df.at[index, 'Objective'] = objective
                continue
    # Save the updated DataFrame to a new Excel file
    df.to_excel(excel_file, index=False)

# Define file paths

# Call the function


In [ ]:
directory =r"D:\Dissertation-GPT\5.25-NhuY\Matcha"  # Replace with your folder path
directory_name = ["AddLesson"]
path= r"D:\Dissertation-GPT\5.25-NhuY\Matcha\summary test case"
create_summary_files(os.path.join(directory,directory_name[0]), path)
create_json_testcase_file(os.path.join(path,directory_name[0])+".txt",os.path.join(path,directory_name[0])+".json")
create_json_testcase_file(os.path.join(path,directory_name[0])+"-Raw.txt",os.path.join(path,directory_name[0])+"-Raw.json")

usecase= "AddLesson"
json_file = os.path.join(path,usecase)+"-Raw.json"
excel_file = r"D:\Dissertation-GPT\code\thu\5.16\output.xlsx"

update_excel_with_json_testcase_data(json_file, excel_file)